In [193]:
import pandas as pd
dataset_spotify = pd.read_csv('data.csv', header=0)
dataset_weather_alexandria = pd.read_csv('alexandria.csv', header=0)

In [194]:
merged_dataset = dataset_spotify.merge(dataset_weather_alexandria, on='date',)
merged_dataset.to_csv('merged.csv', index=False)

In [195]:
def binary_precipitation(value):
    if value != 0:
        return 1
    else:
        return 0


In [196]:
merged_dataset['precipcover'] = merged_dataset['precipcover'].apply(binary_precipitation)


In [197]:
x = merged_dataset.drop(columns=['track_id','playlist_id','date','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','id','duration_ms','time_signature','neg','neu','pos','compound','name','conditions','description','icon','stations','preciptype','sunrise','sunset'])
y = merged_dataset['energy']

x.to_csv('cleaned.csv', index=False)
x.head()

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,moonphase
0,30.0,11.6,21.5,28.1,11.6,21.0,1.9,31.9,0.000,0,...,13.1,348.4,1011.0,12.7,16.0,264.5,22.7,8,10,0.59
1,28.2,15.4,22.3,27.2,15.4,22.2,8.4,42.5,0.000,0,...,25.1,283.7,1008.3,77.8,13.4,226.7,19.5,7,10,0.59
2,31.1,16.6,23.2,29.8,16.6,23.0,13.7,58.3,0.592,100,...,16.5,32.8,1011.1,41.4,16.0,209.8,18.3,7,30,0.81
3,25.9,20.0,23.7,25.9,20.0,23.7,17.6,69.0,5.364,100,...,24.3,202.2,1005.0,86.9,15.5,101.6,8.6,3,30,0.80
4,31.1,16.6,23.2,29.8,16.6,23.0,13.7,58.3,0.592,100,...,16.5,32.8,1011.1,41.4,16.0,209.8,18.3,7,30,0.81


In [198]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [199]:
import tensorflow.keras as keras
model = keras.Sequential()

In [200]:
model.add(keras.layers.Dense(256, input_shape=x_train.shape[1:], activation = 'sigmoid'))
model.add(keras.layers.Dense(256, activation='sigmoid'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [201]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [202]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
17/17 [==============================] - 0s 1ms/step - loss: 0.7001 - accuracy: 0.0000e+00
Epoch 2/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6902 - accuracy: 0.0000e+00
Epoch 3/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6929 - accuracy: 0.0000e+00
Epoch 4/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6922 - accuracy: 0.0000e+00
Epoch 5/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6906 - accuracy: 0.0000e+00
Epoch 6/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6873 - accuracy: 0.0000e+00
Epoch 7/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6877 - accuracy: 0.0000e+00
Epoch 8/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6898 - accuracy: 0.0000e+00
Epoch 9/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6883 - accuracy: 0.0000e+00
Epoch 10/10
17/17 [==============================] - 0s 1ms/step - loss: 0.6914 - 

In [191]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss:.2f}, Accuracy: {accuracy:.2f}")

5/5 [==============================] - 0s 1ms/step - loss: 0.6990 - accuracy: 0.0000e+00
Loss: 0.70, Accuracy: 0.00
